In [ ]:
# 04_extract_kmers.py

import numpy as np
from collections import Counter
from pathlib import Path

# Resolve paths relative to the repository root so the script works regardless
# of current working directory when executed.
PROJECT_ROOT = Path(__file__).resolve().parents[1]
INPUT_PATH = PROJECT_ROOT / "data" / "processed" / "sequences_cleaned.npy"
OUTPUT_PATH = PROJECT_ROOT / "data" / "processed" / "X_kmer.npy"

k = 3

def get_kmers(seq, k):
    return [seq[i:i+k] for i in range(len(seq)-k+1)]

if not INPUT_PATH.exists():
    raise FileNotFoundError(f"Input file not found at {INPUT_PATH}. Run the cleaning step first (02_clean_sequences.py).")

sequences = np.load(str(INPUT_PATH), allow_pickle=True)
if len(sequences) == 0:
    raise ValueError(f"No sequences found in {INPUT_PATH}; nothing to process.")

vocab = set()
for seq in sequences:
    vocab.update(get_kmers(seq, k))

# Sort the vocabulary for deterministic ordering
vocab = sorted(vocab)

def kmer_vector(seq):
    counts = Counter(get_kmers(seq, k))
    return [counts.get(kmer, 0) for kmer in vocab]

X_kmer = np.array([kmer_vector(seq) for seq in sequences])

# Ensure the output directory exists
OUTPUT_PATH.parent.mkdir(parents=True, exist_ok=True)

np.save(str(OUTPUT_PATH), X_kmer)

print("XGBoost input saved with shape:", X_kmer.shape)
